<a href="https://colab.research.google.com/github/tlsgptj/ingongginung_2/blob/main/%EC%9D%B8%EA%B3%B5%EB%8A%A5%EC%A7%80_%EA%B8%B0%EB%A7%90%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

(HW#2) 교재 13장 6절의 다층퍼셉트론 프로그램을 이용하여 UCI의 iris data(붓꽃데이터)를 분류하는 신경망을 완성하시오.

★ 과제 1처럼 PDF 보고서 파일과 소스코드 파일을 제출해 주세요! (양식이 잘못된 경우 감점될 수 있습니다)

★ 시스템상 파일 1개만 업로드되므로, 압축(zip)하여 올려주세요!

★ 평가 기준 등은 과제 2 설명 동영상을 참고해 주세요!

★ 소스코드 파일(mlp.py)로부터 시작하면 조금 더 편리합니다.





- 입력층 노드수: 4

- 은닉층 노드수: 5

- 출력층 노드수: 3



(1) sklearn 라이브러리 활용하여 붓꽃데이터 읽어들임 - datasets.load_iris()

(2) iris data에는 붓꽃샘플이 3가지 클래스 각각에 대해 50개씩, 총 150개의 샘플이 포함되어 있음

(3) 각각의 샘플은 4개의 실수값으로 구성된 4차원 벡터

(4) iris data를 훈련용 120개, 테스트용 30개로 구분하여 사용하고, 훈련데이터에 대한 학습결과와 테스트용 데이터 30개에 대한 분류성능 측정

(5) 신경망의 은닉층 및 출력층 노드에서는 바이어스(bias)를 사용하고, 활성화함수로 sigmoid 함수를 사용

(6) BP알고리즘에서 최대 epoch는 50회, 학습율은 0.01을 사용

(7) 다층퍼셉트론 신경망에 대한 훈련의 결과로 얻어진 모든 weight와 bias 값을 출력 (random하게 초기화)

(8) Python 소스코드와 학습결과에 대한 분석을 정리하여 리포트로 제출

In [ ]:
from sklearn.datasets import load_iris

In [ ]:
import pandas as pd
#붓꽃 데이터 로드
iris = load_iris()
print(iris)

In [ ]:
iris_df = pd.DataFrame(data=iris.data, columns = iris.feature_names)
iris_df['species'] = iris.target

print(iris_df.tail(50))
#품종 전처리는 이미 되어있는 것 같음 0, 1, 2로

In [ ]:
X = iris.data
y = iris.target

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def sigmoid(x):
  return 1/(1+np.exp(-x))

def sigmoid_deriv(out):
  return out*(1-out)

#출력층 활성화 함수(softmax)
def softmax(x):
  #안정적인 softmax 구현
  c = np.max(x, axis=1,keepdims = True)
  exp_x = np.exp(x-c)
  return exp_x / np.sum(exp_x, axis=1, keepdims =True)

#Cross-entropy Loss
def cross_entropy(pred, target):
    # pred: N x 3, target: N x 3 (one-hot)
    # log(0)이 발생하지 않게 작은 값을 더해줌
    return -np.sum(target * np.log(pred + 1e-9)) / len(target)

inputs = 4
hiddens = 5
outputs = 3

learning_rate = 0.05

# 가중치 초기화
np.random.seed(0)
W1 = np.random.randn(inputs, hiddens)*0.01
B1 = np.zeros((hiddens,))
W2 = np.random.randn(hiddens, outputs)*0.01
B2 = np.zeros((outputs,))


In [ ]:
def predict(x):
    # 순방향 전파
    layer0 = x  # (N, 4)
    Z1 = np.dot(layer0, W1) + B1  # (N, 5)
    layer1 = sigmoid(Z1)          # (N, 5)
    Z2 = np.dot(layer1, W2) + B2  # (N, 3)
    layer2 = softmax(Z2)          # (N, 3)
    return layer0, layer1, layer2

In [ ]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

def fit(epochs=3000):
    global W1, W2, B1, B2

    # Create a OneHotEncoder instance outside the loop
    encoder = OneHotEncoder(sparse_output=False, categories='auto')
    # Fit the encoder to your training labels
    Y_train_onehot = encoder.fit_transform(Y_train.reshape(-1, 1))

    for i in range(epochs):
        layer0, layer1, layer2 = predict(X_train)

        # 오차 계산 (cross-entropy)
        loss = cross_entropy(layer2, Y_train)

        # 역전파
        # layer2_delta: 출력층 gradient (softmax + cross-entropy에서 pred - target)
        # Use the one-hot encoded Y_train here
        layer2_delta = (layer2 - Y_train_onehot) / len(X_train)  # (N, 3)

        # 은닉층 오차
        layer1_error = np.dot(layer2_delta, W2.T) # (N,5)
        layer1_delta = layer1_error * sigmoid_deriv(layer1) # (N,5)

        # 가중치 업데이트
        W2 -= learning_rate * np.dot(layer1.T, layer2_delta) # (5,3)
        B2 -= learning_rate * np.sum(layer2_delta, axis=0)   # (3,)
        W1 -= learning_rate * np.dot(layer0.T, layer1_delta) # (4,5)
        B1 -= learning_rate * np.sum(layer1_delta, axis=0)   # (5,)

        if (i+1) % 500 == 0:
            print(f"Epoch {i+1}/{epochs}, Loss: {loss:.4f}")

In [ ]:
def evaluate(X_data, Y_data):
    _, _, layer2 = predict(X_data)
    pred_classes = np.argmax(layer2, axis=1)
    # Change axis to 0 to find the maximum along the first axis (rows)
    true_classes = np.argmax(Y_data, axis=0)
    accuracy = np.mean(pred_classes == true_classes)
    return accuracy

In [ ]:
!pip install scikit-learn

In [ ]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

def cross_entropy(pred, target):
    # Convert target to one-hot encoding
    encoder = OneHotEncoder(sparse_output=False, categories='auto') # 'auto' to infer categories from data
    target_onehot = encoder.fit_transform(target.reshape(-1, 1)) # Reshape target to 2D for OneHotEncoder

    # pred: N x 3, target: N x 3 (one-hot)
    # log(0)이 발생하지 않게 작은 값을 더해줌
    return -np.sum(target_onehot * np.log(pred + 1e-9)) / len(target)

In [ ]:
fit(epochs=5000)

In [ ]:
train_acc = evaluate(X_train, Y_train)
test_acc = evaluate(X_test, Y_test)
print(f"Train Accuracy: {train_acc*100:.2f}%")
print(f"Test Accuracy: {test_acc*100:.2f}%")